TP Bensafi Sarra

Our dataset is ogbn-arxiv network in which each node is a Computer Science paper on the arxiv and each directed edge represents that a paper cited another. The task is to classify each node into a paper class.

In [ ]:
#INSTALLATION

In [ ]:
#Imports
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cpu.html
!pip install -U scikit-learn
!python --version
!pip install -U ogb
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import pandas as pd
import numpy as np
import torch_geometric
import urllib3
import outdated
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cpu.html
     |████████████████████████████████| 286 kB 2.7 MB/s 
     |████████████████████████████████| 641 kB 1.7 MB/s 
     |████████████████████████████████| 311 kB 3.9 MB/s 
     |████████████████████████████████| 121 kB 11.4 MB/s 
     |████████████████████████████████| 467 kB 5.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=85cf75a63ebca98f7e8364502455b5f4d1b9db82b22589f9efffcc1f69f7808c
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python 3.7.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████

### Informations about graphs
if the graphs are complex we need :
  Adjacency matrix: defines how the nodes are connected to each other in a n by n matrix,
    
  Edge attributes: the value of edge (eg. the distance in meters )

In [ ]:
#
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):

    
    def __init__(self, data, num_classes_end, hidden=256
                 , layers_gcn=4):

        super(GCN, self).__init__()#super it self

        self.num_layers_gcn = layers_gcn
        self.conv1 = GCNConv(data.num_node_features, hidden)#num_node_features  grand ici 128
        #The final output of GCN ais a matrix Z , and the shape is N*F . 
          #N is the number of nodes, 
          #F is the number of output features per node

        #The hidden channels parameter that represents the number of hidden units.
        self.bn1=torch.nn.BatchNorm1d(hidden)         
        #we apply BatchNorm1d to the node feature matrix. This way, you compute 
        #statistics for each feature across all nodes in the batch

        self.conv2= GCNConv(hidden, hidden) #SAGEConv(hidden, hidden,   normalize=False)
        self.bn2 = torch.nn.BatchNorm1d(hidden)
        self.conv3= GCNConv(hidden, hidden) 
        self.bn3 = torch.nn.BatchNorm1d(hidden)

        # couche de sortie
        self.conv4 = GCNConv(hidden, num_classes_end)

    # reset les paramètres 
    def reset_parameters(self):
        
      self.conv1.reset_parameters()
      self.conv2.reset_parameters()
      self.conv3.reset_parameters()
      self.conv4.reset_parameters()
      self.bn1.reset_parameters()
      self.bn2.reset_parameters()
      self.bn3.reset_parameters()
      

    def forward(self, data):
        # Nos données d'entrée 
         # x : les valeurs des  nœuds
         # les nœuds d'origine et de destinations
        x, adj_t = data.x, data.adj_t
        # on ajoute les couches de convolution
        x = self.conv1(x, adj_t)  #initialiser la première couche convolutive de graphe,  
        x = self.bn1(x) 
        #ReLU and a dropout layer for regularization purposes
        x = F.relu(x)#suivie d'une fonction d'activation ReLU
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = self.conv2(x, adj_t)
        x = self.bn2(x) 
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, adj_t)
        x = self.bn3(x) 
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        
        # couche de sortie
        x = self.conv4(x, adj_t)
        
        x = F.log_softmax(x, dim=1)  
        return x



In [ ]:
def train(model, data, train_idx, optimizer):
    model.train() #entrainer le models
    optimizer.zero_grad() #In PyTorch, for every mini-batch during the training phase, we typically want to explicitly set the gradients to zero before starting to do backpropragation (i.e., updating the Weights and biases) because PyTorch accumulates the gradients on subsequent backward passes. This accumulating behaviour is convenient while training RNNs or when we want to compute the gradient of the loss summed over multiple mini-batches. So, the default action has been set to accumulate (i.e. sum) the gradients on every loss.backward() call.
    
    out = model(data)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])#negative log likelihood
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
#Here the data is already splited by the OGB
dataset = PygNodePropPredDataset(name='ogbn-arxiv', transform=T.ToSparseTensor())
print(dataset.num_node_features)
#split data
split_idx = dataset.get_idx_split()
# data
data = dataset[0]#data dans la position zero on a un seul dataset
print(data)
data = data.to(device)
data.adj_t = data.adj_t.to_symmetric()#important pour enlever l'orientation

# **********************************
split_idx = dataset.get_idx_split() 
train_idx = split_idx['train'].to(device)
evaluator = Evaluator(name='ogbn-arxiv')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:16<00:00,  4.78it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 7695.97it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 1934.64it/s]

Saving...



Done!


128
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


In [ ]:
from torch._C import BenchmarkExecutionStats
log_steps = 1
Accvalid = []
Acctest = []
besttest = 0
bestvalid = 0

#Hyperparameters
epochs = 50
hidden_channels = 512
dropout = 0.5
lr = 0.01

model = GCN(data, dataset.num_classes).to(device)
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, 1 + epochs):
  loss = train(model, data, train_idx, optimizer)
  result = test(model, data, split_idx, evaluator)

  if epoch % log_steps == 0:
    train_acc, valid_acc, test_acc = result
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}%')
                  
  
  
  if (bestvalid<valid_acc) : 
      bestvalid=valid_acc
      besttest=test_acc

print(f' Test {besttest}')

Epoch: 01, Loss: 4.0140, Train: 27.91%, Valid: 30.05%
Epoch: 02, Loss: 2.5496, Train: 31.96%, Valid: 40.06%
Epoch: 03, Loss: 2.2060, Train: 33.35%, Valid: 38.26%
Epoch: 04, Loss: 2.0134, Train: 44.19%, Valid: 46.51%
Epoch: 05, Loss: 1.8693, Train: 46.84%, Valid: 45.99%
Epoch: 06, Loss: 1.7406, Train: 48.45%, Valid: 48.12%
Epoch: 07, Loss: 1.6574, Train: 50.76%, Valid: 51.10%
Epoch: 08, Loss: 1.5864, Train: 54.22%, Valid: 55.76%
Epoch: 09, Loss: 1.5268, Train: 55.87%, Valid: 60.03%
Epoch: 10, Loss: 1.4875, Train: 56.42%, Valid: 60.83%
Epoch: 11, Loss: 1.4532, Train: 56.68%, Valid: 61.07%
Epoch: 12, Loss: 1.4188, Train: 57.23%, Valid: 61.45%
Epoch: 13, Loss: 1.3917, Train: 58.38%, Valid: 61.77%
Epoch: 14, Loss: 1.3597, Train: 59.25%, Valid: 62.14%
Epoch: 15, Loss: 1.3304, Train: 59.78%, Valid: 62.37%
Epoch: 16, Loss: 1.3115, Train: 60.42%, Valid: 62.84%
Epoch: 17, Loss: 1.2938, Train: 61.33%, Valid: 63.54%
Epoch: 18, Loss: 1.2751, Train: 62.29%, Valid: 64.00%
Epoch: 19, Loss: 1.2532, Tra